# Visualizations in Plotly for the DMP Aspect Test Experiment

In [1]:
%matplotlib notebook

In [2]:
import sys
sys.path.append("../../")

from dmp.data.logging import _get_sql_engine
import pandas as pd
import gc
import time
import os
import json
import datetime
import sqlalchemy

from os import listdir
from os.path import isfile, join

import numpy as np
import pandas as pd

import plotly
import plotly.express as px

from joblib import Memory

from ipywidgets import interact


In [3]:
db = _get_sql_engine()

memory = Memory(location="./cache", verbose=0)

def query(query_string):
    with db.connect() as engine:
        return pd.read_sql(query_string, engine)
    
@memory.cache
def cached_query(query_string):
    return query(query_string)

def clear_cache():
    memory.clear()

In [4]:
import plotly.express as px

In [5]:
### Heapmap App

def heatmap_app(dataset="537_houses", topology="wide_first", loss="history_val_mean_squared_error", agg="avg"):
    query_string = f'''
select "config.budget", {agg}(a.val) as value, count(a.val), a.epoch
from
    materialized_experiments_0 t,
    unnest(t.{loss}) WITH ORDINALITY as a(val, epoch)
WHERE
    "groupname" IN ('exp00', 'exp01') and
    "config.dataset"='{dataset}' and
    "config.topology"='{topology}' and
    "config.residual_mode"='none'
GROUP BY epoch, "config.budget"
'''
    df = cached_query(query_string)
    img = df.pivot_table(columns="epoch", index="config.budget", values="value")
    img.index = img.index.astype("str") #plotly interprets the y axis as numeric otherwise
    return px.imshow(img)

In [6]:
dataset_names = ['529_pollen',
    'sleep',
    'adult',
    '537_houses',
    'nursery',
    '201_pol',
    'mnist',
    'connect_4',
    'wine_quality_white']

aggregates = ["avg", "min", "max"]

topologies = ["exponential", "rectangle", "trapezoid", "wide_first"]

losses = ['history_loss', 'history_hinge', 'history_accuracy',
       'history_val_loss', 'history_val_hinge', 'history_val_accuracy',
       'history_squared_hinge', 'history_cosine_similarity',
       'history_val_squared_hinge', 'history_mean_squared_error',
       'history_mean_absolute_error', 'history_val_cosine_similarity',
       'history_val_mean_squared_error', 'history_root_mean_squared_error',
       'history_val_mean_absolute_error',
       'history_kullback_leibler_divergence',
       'history_val_root_mean_squared_error',
       'history_mean_squared_logarithmic_error',
       'history_val_kullback_leibler_divergence',
       'history_val_mean_squared_logarithmic_error']

interact(heatmap_app, dataset=dataset_names, agg=aggregates, topology=topologies, loss=losses)

interactive(children=(Dropdown(description='dataset', index=3, options=('529_pollen', 'sleep', 'adult', '537_h…

<function __main__.heatmap_app(dataset='537_houses', topology='wide_first', loss='history_val_mean_squared_error', agg='avg')>

##

537 houses, (val) kl divergence, trapezoid

In [4]:
df = cache_query('''select * from materialized_experiments_0 WHERE "groupname" IN ('exp00', 'exp01') limit 10''')
display(df.columns)
df

Index(['id', 'name', 'timestamp', 'doc', 'job', 'groupname', 'jobid',
       'iterations', 'loss', 'num_classes', 'num_features', 'num_inputs',
       'num_observations', 'num_outputs', 'num_weights', 'run_name', 'task',
       'test_loss', 'val_loss', 'config.activation', 'config.budget',
       'config.dataset', 'config.depth', 'config.early_stopping.baseline',
       'config.early_stopping.min_delta', 'config.early_stopping.mode',
       'config.early_stopping.monitor', 'config.early_stopping.patience',
       'config.log', 'config.mode', 'config.name', 'config.num_hidden',
       'config.residual_mode', 'config.run_name', 'config.test_split',
       'config.topology', 'history_loss', 'history_hinge', 'history_accuracy',
       'history_val_loss', 'history_val_hinge', 'history_val_accuracy',
       'history_squared_hinge', 'history_cosine_similarity',
       'history_val_squared_hinge', 'history_mean_squared_error',
       'history_mean_absolute_error', 'history_val_cosine_similarit

,id,name,timestamp,doc,job,groupname,jobid,iterations,loss,num_classes,...,history_mean_absolute_error,history_val_cosine_similarity,history_val_mean_squared_error,history_root_mean_squared_error,history_val_mean_absolute_error,history_kullback_leibler_divergence,history_val_root_mean_squared_error,history_mean_squared_logarithmic_error,history_val_kullback_leibler_divergence,history_val_mean_squared_logarithmic_error
0,461287,mnist__rectangle__33554432__10__16251698128493,2021-07-01 17:21:37.506796,"{'loss': 0.02577941305935383, 'task': 'classif...",067bb44f-eb36-4139-8688-0590eb966bde,exp01,None,9,0.025779,10,...,"[0.06908129155635834, 0.012601211667060852, 0....","[0.9551001787185669, 0.9690148830413818, 0.971...","[0.007836131379008293, 0.005475647747516632, 0...","[0.18842794001102448, 0.07946979254484177, 0.0...","[0.01633361168205738, 0.01052351389080286, 0.0...","[1.0395499467849731, 0.13780289888381958, 0.08...","[0.08852192014455795, 0.0739976167678833, 0.07...","[0.017356442287564278, 0.003071459708735347, 0...","[0.172026127576828, 0.12255849689245224, 0.112...","[0.0038135191425681114, 0.0026676489505916834,..."
1,461288,mnist__rectangle__33554432__3__16251695301180,2021-07-01 17:22:19.334815,"{'loss': 0.044262394309043884, 'task': 'classi...",0ddc1fe0-4a2f-4adf-a8b6-0a85648d9ca1,exp01,None,3,0.044262,10,...,"[0.019615791738033295, 0.006769857835024595, 0...","[0.96994948387146, 0.9756336212158203, 0.97980...","[0.005368464160710573, 0.004355143290013075, 0...","[0.09862419962882996, 0.05802123621106148, 0.0...","[0.01040168572217226, 0.0076765986159443855, 0...","[0.22742772102355957, 0.06924095749855042, 0.0...","[0.0732698142528534, 0.06599351018667221, 0.06...","[0.004756750538945198, 0.0016424678033217788, ...","[0.11719939857721329, 0.0935586541891098, 0.08...","[0.0026163365691900253, 0.0021229723934084177,..."
2,461289,mnist__rectangle__33554432__7__16251696611450,2021-07-01 17:22:23.247860,"{'loss': 0.038759976625442505, 'task': 'classi...",e19d0b02-9819-4274-8727-a50339d15ce1,exp01,None,5,0.038760,10,...,"[0.03776571527123451, 0.009435620158910751, 0....","[0.9616579413414001, 0.969774603843689, 0.9718...","[0.006850193254649639, 0.005432771053165197, 0...","[0.13962528109550476, 0.06856178492307663, 0.0...","[0.01455457229167223, 0.009346023201942444, 0....","[0.5787925720214844, 0.10105635225772858, 0.06...","[0.08276589214801788, 0.07370731234550476, 0.0...","[0.009535985998809338, 0.002282999688759446, 0...","[0.14887908101081848, 0.12380843609571457, 0.1...","[0.0033194092102348804, 0.002639091107994318, ..."
3,461290,mnist__rectangle__33554432__3__16251694816276,2021-07-01 17:23:11.670714,"{'loss': 0.025658192113041878, 'task': 'classi...",0703e8a7-a026-4821-b63a-80b192449193,exp01,None,5,0.025658,10,...,"[0.019375476986169815, 0.006779802497476339, 0...","[0.9705445170402527, 0.9764423370361328, 0.974...","[0.0051925028674304485, 0.004199092276394367, ...","[0.09816713631153107, 0.05797600373625755, 0.0...","[0.009827564470469952, 0.0072421422228217125, ...","[0.22534804046154022, 0.06952840834856033, 0.0...","[0.0720590278506279, 0.06480039656162262, 0.06...","[0.0047170622274279594, 0.001638107351027429, ...","[0.11248726397752762, 0.09395160526037216, 0.0...","[0.0025376183912158012, 0.002046099165454507, ..."
4,461291,mnist__rectangle__33554432__10__16251699652625,2021-07-01 17:27:36.166244,"{'loss': 0.05591704323887825, 'task': 'classif...",14a65369-f3c1-47b8-a269-283638c04d25,exp01,None,5,0.055917,10,...,"[0.06900423765182495, 0.013961142860352993, 0....","[0.9458900094032288, 0.9633675813674927, 0.969...","[0.009430465288460255, 0.006462868303060532, 0...","[0.1875855177640915, 0.08357872068881989, 0.06...","[0.019188109785318375, 0.011685683391988277, 0...","[1.046220064163208, 0.15162283182144165, 0.101...","[0.09711059927940369, 0.08039195835590363, 0.0...","[0.01725294627249241, 0.0034010959789156914, 0...","[0.20569998025894165, 0.14036983251571655, 0.1...","[0.0045869858004152775, 0.0

In [38]:
# Plot the heatmap of validation loss for one topology, one dataset, over iteration count and different budgets

q = '''
select "config.budget", avg(a.val) as val_loss, avg(a.loss) as train_loss, count(a.loss), a.epoch
from
    materialized_experiments_0 t,
    unnest(t.history_val_mean_squared_error, t.history_mean_squared_error) WITH ORDINALITY as a(val, loss, epoch)
WHERE
    "groupname" IN ('exp00', 'exp01') and
    "config.dataset"='537_houses' and
    "config.topology"='wide_first' and
    "config.residual_mode"='none'
GROUP BY epoch, "config.budget"
'''

# q = '''
# select distinct("name")
# from
#     materialized_experiments_0
# WHERE
#     "groupname" IN ('exp00', 'exp01') and
#     "config.dataset"='537_houses' and
#     "config.topology"='wide_first' and
#     "config.residual_mode"='none' and
#     "config.budget"=262144
# limit 10
# '''

df = cache_query(q)
display(df.columns)
df

Index(['config.budget', 'val_loss', 'train_loss', 'count', 'epoch'], dtype='object')

,config.budget,val_loss,train_loss,count,epoch
0,32,0.064403,0.066617,390,1
1,64,0.063871,0.065595,390,1
2,128,0.063801,0.066003,390,1
3,256,0.063158,0.064527,390,1
4,512,0.061834,0.063143,390,1
...,...,...,...,...,...
1662,32,0.016816,0.017276,1,197
1663,32,0.016810,0.017291,1,198
1664,32,0.016820,0.017257,1,199
1665,32,0.016770,0.017270,1,200
